In [1]:
!sudo apt install swi-prolog

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  autopoint debhelper debugedit dh-autoreconf dh-strip-nondeterminism dwz
  gettext gettext-base intltool-debian libarchive-cpio-perl
  libarchive-zip-perl libdebhelper-perl libfile-stripnondeterminism-perl
  libmail-sendmail-perl libossp-uuid16 libsub-override-perl
  libsys-hostname-long-perl libtool po-debconf swi-prolog-core
  swi-prolog-core-packages swi-prolog-doc swi-prolog-nox swi-prolog-x
Suggested packages:
  dh-make gettext-doc libasprintf-dev libgettextpo-dev uuid libtool-doc
  gcj-jdk libmail-box-perl elpa-ediprolog swi-prolog-java swi-prolog-odbc
  swi-prolog-bdb
The following NEW packages will be installed:
  autopoint debhelper debugedit dh-autoreconf dh-strip-nondeterminism dwz
  gettext gettext-base intltool-debian libarchive-cpio-perl
  libarchive-zip-perl libdebhelper-perl libfile-stripnondeterminism-perl
  libmail-send

In [2]:
!pip install pyswip

In [3]:
import pandas as pd
import numpy as np

In [17]:
# Loading the data and renaming the columns
# The names now correspond to the statement types

data = pd.read_csv('/content/CS152 LBA data - Sheet1.csv', skiprows=1)
col_names = ['name', 'hasCuisine', 'priceRange', 'mealTypes', 'distance',
             'isVegetarian', 'glutenFree', 'hasLanguage', 'transportation']
data.columns = col_names

In [18]:
data.head(5)

,name,hasCuisine,priceRange,mealTypes,distance,isVegetarian,glutenFree,hasLanguage,transportation
0,Maroush,"Lebanese, Middle Eastern",low,"Lunch, Dinner",9,Y,Unknown,"German, English, Arabic",Walk
1,Burgermeister Kottbusser Tor,American,low,"Lunch, Dinner",12,Y,Unknown,"German, English",Walk
2,Basmah,"Sudanese, Mediterranean",low,"Lunch, Dinner",15,Y,Unknown,"German, English, Arabic","Walk, Bus"
3,Mundvoll,International,medium,"Breakfast, Brunch, Lunch, Dinner",6,Y,Y,"German, English",Walk
4,Paglia,"Italian, Mediterranean",medium,"Brunch, Lunch, Dinner",9,Y,Unknown,"German, English",Walk


### Building the KB

In [20]:
# Converted Euro signs to three price levels: low, medium, high in Excel

# all_commands = []
listCommands = []

for k in range(len(data)):
  row = data.loc[k, :]
  resName = row[0]
  # concatenate words in the name, converted to lowercase (can fix to join with lower dash later)
  resName = ''.join(e for e in resName.lower() if e.isalnum())
  # listCommands = [] # all commands per single restaurant name
  for i in range(1, len(row)):
    category = data.columns[i]
    items = row[i]
    if type(items) == str:
      items = row[i].split(', ')
      for item in items:
        item = ''.join(e for e in item.lower() if e.isalnum())
        command = "{0}({1}, {2})".format(category, resName, item)
        listCommands.append(command)
    else:
        command = "{0}({1}, {2})".format(category, resName, items)
        listCommands.append(command)
  # all_commands.append(listCommands)

In [21]:
# Checking to see how Middle eastern got converted
print(listCommands[12:14])

# Then write all the clauses to a text file for inspection
with open('/content/kb.pl', 'w') as file:
    # Iterate through each item in the array
    for item in listCommands:
        # Write the item to the file followed by a newline character
        file.write(item + '.\n')

['hasCuisine(burgermeisterkottbussertor, american)', 'priceRange(burgermeisterkottbussertor, low)']


In [22]:
from pyswip import Prolog
prolog = Prolog()

# Assert KB inputs for all commands
for command in listCommands:
  prolog.assertz(command)

### Queries

In [23]:
# Includes all the variables apart from Language
prolog.assertz("query(X, Cuisine, Price, Distance, Vegetarian, Mealtype, Glutenfree, Language, Transport) :- hasCuisine(X, Cuisine), priceRange(X, Price), distance(X, Y), Y < Distance, isVegetarian(X, Vegetarian), glutenFree(X, Glutenfree), mealTypes(X, Mealtype), hasLanguage(X, Language), transportation(X, Transport)")

In [42]:
# Latest update: changed distance parameter default value to 1000 from -1

# search wrapper breaks down the attributes that have multiple values
# It implements the logical operation OR
def searchWrapper(**attributes):

  # initialize dictionary for single and multiple arguments
  d_single, d_multi = {}, {}
  to_query = []

  for k, v in attributes.items():
    if ',' in v:
      # if there are multiple arguments record them in a query
      d_multi[k] = v.split(',')
      to_query.append(k)
    else:
      d_single[k] = v

  # use backtracking to find all combinations
  def runAllCombination(query):
    if not query: return [{}] # base case: return empty dictionary

    p = query[0]
    res = runAllCombination(query[1:])
    fin = []

    # update new dictionaries with all possible combinations
    for arg in d_multi[p]:
      for pos in res:
        temp = dict(pos)
        temp.update({p: arg})
        fin.append(temp)

    return fin

  result = set()

  # run through all possible combinations and update result
  for m in runAllCombination(to_query):
    m.update(d_single)
    result.update(search_new(**m))

  return result

def search_new(cuisine = '_', price = '_', distance = '1000', isvegetarian = '_', mealtype = '_', isglutenfree = '_', language = '_' , transport = '_'):
  to_query = "query(X, %s)" % (",".join([cuisine, price, distance, isvegetarian, mealtype, isglutenfree, language, transport]))
  print("to_query = " + to_query)
  try:
    return set([soln["X"] for soln in prolog.query(to_query)])
  except:
    return set()

In [43]:
# searchWrapper(**attributes) calls search_new(...)
print( 'searchWrapper call:')
print (searchWrapper(cuisine = 'lebanese, middleeastern, thai, italian') )

searchWrapper call:
to_query = query(X, lebanese,_,1000,_,_,_,_,_)
to_query = query(X,  middleeastern,_,1000,_,_,_,_,_)
to_query = query(X,  thai,_,1000,_,_,_,_,_)
to_query = query(X,  italian,_,1000,_,_,_,_,_)
{'maroush', 'lemongrass', 'lapiadina', 'paglia'}


In [44]:
# search_new implements AND, as defined above in the Prolog rule
print( search_new(cuisine = 'lebanese, middleeastern, thai, italian') )
print( search_new(cuisine = 'italian') )

to_query = query(X, lebanese, middleeastern, thai, italian,_,1000,_,_,_,_,_)
set()
to_query = query(X, italian,_,1000,_,_,_,_,_)
{'lapiadina', 'paglia'}


In [45]:
search_new(distance = '15')

to_query = query(X, _,_,15,_,_,_,_,_)


{'burgermeisterkottbussertor',
 'hasirburger',
 'lapiadina',
 'longmarchcanteen',
 'maroush',
 'milchzucker',
 'mundvoll',
 'paglia',
 'quepasamexicana',
 'santamaria',
 'sushiforyou'}

### Test cases

In [46]:
# Search case 1: All restaurants within a 15 min walking distance
searchWrapper(distance = '15')

to_query = query(X, _,_,15,_,_,_,_,_)


{'burgermeisterkottbussertor',
 'hasirburger',
 'lapiadina',
 'longmarchcanteen',
 'maroush',
 'milchzucker',
 'mundvoll',
 'paglia',
 'quepasamexicana',
 'santamaria',
 'sushiforyou'}

In [47]:
# Search case 2: A restaurant within a 10 min walking distance in a low price range
# that has both English-speaking and German-speaking staff, vegetarian and dinner options
searchWrapper(distance = "10", price="low", isvegetarian="y", mealtype="dinner", language = 'english, german', transport="walk")

to_query = query(X, _,low,10,y,dinner,_,english,walk)
to_query = query(X, _,low,10,y,dinner,_, german,walk)


{'lapiadina', 'maroush', 'santamaria'}

In [48]:
# Search case 3: Filter by cuisine (italian, vietnamese) and price (medium)
searchWrapper(cuisine = 'italian, vietnamese', price="medium")

to_query = query(X, italian,medium,1000,_,_,_,_,_)
to_query = query(X,  vietnamese,medium,1000,_,_,_,_,_)


{'eden', 'paglia'}

In [49]:
# Search case 4: Group dinner search; prefer American cuisine in a low price range and near the residence
searchWrapper(cuisine = 'american', distance = "15", price="low")

to_query = query(X, american,low,15,_,_,_,_,_)


{'burgermeisterkottbussertor'}

In [50]:
from ipywidgets import interact
interact(searchWrapper, cuisine = "_", price = "_", distance = "1000", isvegetarian = "_", mealtype = '_', isglutenfree = '_', transport = '_')
# interact(search, cuisine = "_", price = "_", distance = "-1")

interactive(children=(Text(value='_', description='cuisine'), Text(value='_', description='price'), Text(value…

<function __main__.searchWrapper(**attributes)>